In [1]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
import keras.backend as K
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

In [2]:
# Store the input data in a dataframe
conflict_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_Java.csv")

#Show the data
conflict_data.head()

,parallel_changed_file_num,commit_num,file_added,file_removed,file_renamed,file_modified,file_copied,line_added,line_removed,developer_num,...,fix_frequency,improve_frequency,refactor_frequency,remove_frequency,update_frequency,use_frequency,messages_min,messages_max,messages_mean,messages_median
0,0,1,0,3,0,0,0,0,306,1,...,0,0,0,1,0,0,17,17,17.000000,17.0
1,0,15,-11,0,0,-80,0,-759,-254,-3,...,4,0,0,1,0,2,16,133,52.466667,48.0
2,0,3,-2,1,0,3,0,-24,16,0,...,0,0,0,0,1,0,42,62,49.333333,44.0
3,0,2,-7,-11,0,-39,0,-2316,-3378,0,...,0,0,0,0,1,0,17,42,29.500000,29.5
4,0,39,-65,-16,0,-230,0,-9213,-4131,-8,...,4,2,0,2,0,1,0,65,45.904762,45.0


In [3]:
# create an abs_scaler object
abs_scaler = MaxAbsScaler()

# calculate the maximum absolute value for scaling the data using the fit method
abs_scaler.fit(conflict_data)

# transform the data using the parameters calculated by the fit method (the maximum absolute values)
scaled_data = abs_scaler.transform(conflict_data)

# store the results in a data frame
df_scaled = pd.DataFrame(scaled_data, columns=conflict_data.columns)

# visualize the data frame
df_scaled

,parallel_changed_file_num,commit_num,file_added,file_removed,file_renamed,file_modified,file_copied,line_added,line_removed,developer_num,...,fix_frequency,improve_frequency,refactor_frequency,remove_frequency,update_frequency,use_frequency,messages_min,messages_max,messages_mean,messages_median
0,0.000000,0.000119,0.000000,0.000124,0.0,0.000000,0.0,0.000000e+00,1.298694e-05,0.002833,...,0.000000,0.000000,0.0,0.052632,0.000000,0.000000,0.0425,0.041667,0.042500,0.042394
1,0.000000,0.001785,-0.000327,0.000000,0.0,-0.000575,0.0,-4.322792e-05,-1.078001e-05,-0.008499,...,0.210526,0.000000,0.0,0.052632,0.000000,0.086957,0.0400,0.325980,0.131167,0.119701
2,0.000000,0.000357,-0.000059,0.000041,0.0,0.000022,0.0,-1.366891e-06,6.790559e-07,0.000000,...,0.000000,0.000000,0.0,0.000000,0.030303,0.000000,0.1050,0.151961,0.123333,0.109726
3,0.000000,0.000238,-0.000208,-0.000454,0.0,-0.000280,0.0,-1.319050e-04,-1.433657e-04,0.000000,...,0.000000,0.000000,0.0,0.000000,0.030303,0.000000,0.0425,0.102941,0.073750,0.073566
4,0.000000,0.004642,-0.001931,-0.000660,0.0,-0.001654,0.0,-5.247152e-04,-1.753237e-04,-0.022663,...,0.210526,0.181818,0.0,0.105263,0.000000,0.043478,0.0000,0.159314,0.114762,0.112219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26694,0.000000,0.002618,0.000000,0.000000,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,-0.002833,...,0.157895,0.000000,0.0,0.157895,0.000000,0.000000,0.0450,0.578431,0.170714,0.089776
26695,0.000000,0.000238,0.000030,0.000248,0.0,0.000043,0.0,7.802668e-06,1.065269e-05,0.002833,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.1175,0.115196,0.117500,0.117207
26696,0.000000,0.002499,-0.001960,-0.002022,0.0,-0.001381,0.0,-4.562568e-04,-2.978509e-04,-0.014164,...,0.157895,0.000000,0.0,0.052632,0.090909,0.000000,0.0300,0.379902,0.151071,0.118454
26697,0.000089,0.001904,-0.000267,-0.000083,0.0,-0.000475,0.0,-2.311185e-04,-1.001607e-05,-0.005666,...,0.000000,0.000000,0.0,0.052632,0.000000,0.000000,0.0475,0.132353,0.091250,0.091022


In [4]:
conflict_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_Java.csv")

#Show the data
conflict_labels.head()

,is_conflict
0,0
1,0
2,0
3,0
4,0


In [5]:
#Append labels to the training data 
df_scaled.insert(28, 'is_conflict', conflict_labels)

df_scaled

,parallel_changed_file_num,commit_num,file_added,file_removed,file_renamed,file_modified,file_copied,line_added,line_removed,developer_num,...,improve_frequency,refactor_frequency,remove_frequency,update_frequency,use_frequency,messages_min,messages_max,messages_mean,messages_median,is_conflict
0,0.000000,0.000119,0.000000,0.000124,0.0,0.000000,0.0,0.000000e+00,1.298694e-05,0.002833,...,0.000000,0.0,0.052632,0.000000,0.000000,0.0425,0.041667,0.042500,0.042394,0
1,0.000000,0.001785,-0.000327,0.000000,0.0,-0.000575,0.0,-4.322792e-05,-1.078001e-05,-0.008499,...,0.000000,0.0,0.052632,0.000000,0.086957,0.0400,0.325980,0.131167,0.119701,0
2,0.000000,0.000357,-0.000059,0.000041,0.0,0.000022,0.0,-1.366891e-06,6.790559e-07,0.000000,...,0.000000,0.0,0.000000,0.030303,0.000000,0.1050,0.151961,0.123333,0.109726,0
3,0.000000,0.000238,-0.000208,-0.000454,0.0,-0.000280,0.0,-1.319050e-04,-1.433657e-04,0.000000,...,0.000000,0.0,0.000000,0.030303,0.000000,0.0425,0.102941,0.073750,0.073566,0
4,0.000000,0.004642,-0.001931,-0.000660,0.0,-0.001654,0.0,-5.247152e-04,-1.753237e-04,-0.022663,...,0.181818,0.0,0.105263,0.000000,0.043478,0.0000,0.159314,0.114762,0.112219,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26694,0.000000,0.002618,0.000000,0.000000,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,-0.002833,...,0.000000,0.0,0.157895,0.000000,0.000000,0.0450,0.578431,0.170714,0.089776,0
26695,0.000000,0.000238,0.000030,0.000248,0.0,0.000043,0.0,7.802668e-06,1.065269e-05,0.002833,...,0.000000,0.0,0.000000,0.000000,0.000000,0.1175,0.115196,0.117500,0.117207,0
26696,0.000000,0.002499,-0.001960,-0.002022,0.0,-0.001381,0.0,-4.562568e-04,-2.978509e-04,-0.014164,...,0.000000,0.0,0.052632,0.090909,0.000000,0.0300,0.379902,0.151071,0.118454,0
26697,0.000089,0.001904,-0.000267,-0.000083,0.0,-0.000475,0.0,-2.311185e-04,-1.001607e-05,-0.005666,...,0.000000,0.0,0.052632,0.000000,0.000000,0.0475,0.132353,0.091250,0.091022,0


In [6]:
# Split the data into training (80%) and testing (20%)
train, test = train_test_split(df_scaled, test_size=0.2)

# Split data into features and labels (only using the first 29 columns as 30 is the amount and 31 is the label
train_features = np.array(train.values[:,:27])
train_labels = np.array(train.values[:,-1])
test_features = np.array(test.values[:,:27])
test_labels = np.array(test.values[:,-1])

In [86]:
model = Sequential()
model.add(Dense(units=100, kernel_initializer='uniform', input_dim=train_features.shape[1], activation='tanh'))
model.add(Dropout(.2))
model.add(Dense(units=100, kernel_initializer='uniform', activation='tanh'))
model.add(Dropout(.2))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [87]:
train_history = model.fit(train_features, train_labels, batch_size=50, epochs=200, verbose=1, validation_split=0.2)

Epoch 1/200
342/342 [==============================] - 1s 1ms/step - loss: 0.2972 - accuracy: 0.9170 - val_loss: 0.2503 - val_accuracy: 0.9270
Epoch 2/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2574 - accuracy: 0.9225 - val_loss: 0.2483 - val_accuracy: 0.9265
Epoch 3/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2548 - accuracy: 0.9228 - val_loss: 0.2462 - val_accuracy: 0.9277
Epoch 4/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2535 - accuracy: 0.9235 - val_loss: 0.2468 - val_accuracy: 0.9279
Epoch 5/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2527 - accuracy: 0.9232 - val_loss: 0.2480 - val_accuracy: 0.9279
Epoch 6/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2520 - accuracy: 0.9233 - val_loss: 0.2446 - val_accuracy: 0.9279
Epoch 7/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2513 - accuracy: 0.9234 - val_loss: 0.2451 - val_accuracy: 0.9281

Epoch 58/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2037 - accuracy: 0.9244 - val_loss: 0.1940 - val_accuracy: 0.9307
Epoch 59/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2040 - accuracy: 0.9234 - val_loss: 0.1959 - val_accuracy: 0.9312
Epoch 60/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2026 - accuracy: 0.9253 - val_loss: 0.1930 - val_accuracy: 0.9314
Epoch 61/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2026 - accuracy: 0.9225 - val_loss: 0.1906 - val_accuracy: 0.9316
Epoch 62/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2004 - accuracy: 0.9254 - val_loss: 0.1943 - val_accuracy: 0.9307
Epoch 63/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2031 - accuracy: 0.9237 - val_loss: 0.1931 - val_accuracy: 0.9307
Epoch 64/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1988 - accuracy: 0.9241 - val_loss: 0.1898 - val_accuracy:

Epoch 115/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1725 - accuracy: 0.9344 - val_loss: 0.1610 - val_accuracy: 0.9408
Epoch 116/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1716 - accuracy: 0.9334 - val_loss: 0.1602 - val_accuracy: 0.9396
Epoch 117/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1731 - accuracy: 0.9328 - val_loss: 0.1736 - val_accuracy: 0.9382
Epoch 118/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1718 - accuracy: 0.9342 - val_loss: 0.1645 - val_accuracy: 0.9398
Epoch 119/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1691 - accuracy: 0.9345 - val_loss: 0.1596 - val_accuracy: 0.9401
Epoch 120/200
342/342 [==============================] - 0s 1ms/step - loss: 0.1687 - accuracy: 0.9351 - val_loss: 0.1649 - val_accuracy: 0.9426
Epoch 121/200
342/342 [==============================] - 1s 2ms/step - loss: 0.1709 - accuracy: 0.9338 - val_loss: 0.1578 - val_ac

In [88]:
# Get model accuracy 
scores = model.evaluate(test_features, test_labels)
print('\n')
print('accuracy=',scores[1])

# Get predictions from model
output = model.predict_classes(test_features)

# Show confusion matrix
y_actu = pd.Series(test_labels, name='Actual')
y_pred = pd.Series(np.ndarray.flatten(output), name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

num_positives =  (np.count_nonzero(y_actu))
num_negatives = y_actu.size - num_positives

TN = df_confusion[0][0]
TP = df_confusion[1][1]
FN = df_confusion[0][1]
FP = df_confusion[1][0]

precision = TP/(TP + FP)
recall = TP/(TP + FN)
specificity = TN/(TN + FP)
f1 = 2*((precision*recall)/(precision+recall))
print("Num Fraud: {}".format(num_positives))

print("Precision: ", precision)
print("Recall/Sensitivity: ", recall)
print("Specificity: ", specificity)
print("F1 Score: ", f1)

167/167 [==============================] - 0s 528us/step - loss: 0.1374 - accuracy: 0.9448


accuracy= 0.9447565674781799
Predicted     0    1
Actual              
0.0        4809  108
1.0         187  236
Num Fraud: 423
Precision:  0.686046511627907
Recall/Sensitivity:  0.557919621749409
Specificity:  0.9780353874313605
F1 Score:  0.6153846153846154


In [24]:
#Cost sensitive
class_weights = class_weight.compute_class_weight({0:1, 1:5}, np.unique(train_labels), train_labels)
class_weights = {i : class_weights[i] for i in range(2)}

C:\Users\victoriawu\Anaconda3\envs\MSFTHackathon2021\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [27]:
model2 = Sequential()
model2.add(Dense(units=100, kernel_initializer='uniform', input_dim=train_features.shape[1], activation='relu'))
model2.add(Dropout(.2))
model2.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model2.add(Dropout(.2))
model2.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model2.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [28]:
train_history2 = model2.fit(train_features, train_labels, batch_size=50, epochs=200, verbose=1, validation_split=0.2, class_weight=class_weights)

Epoch 1/200
342/342 [==============================] - 0s 1ms/step - loss: 0.7848 - accuracy: 0.9188 - val_loss: 0.3638 - val_accuracy: 0.9209
Epoch 2/200
342/342 [==============================] - 0s 1ms/step - loss: 0.7245 - accuracy: 0.9180 - val_loss: 0.3701 - val_accuracy: 0.9174
Epoch 3/200
342/342 [==============================] - 0s 1ms/step - loss: 0.6983 - accuracy: 0.9175 - val_loss: 0.4112 - val_accuracy: 0.9118
Epoch 4/200
342/342 [==============================] - 0s 1ms/step - loss: 0.6769 - accuracy: 0.9152 - val_loss: 0.3709 - val_accuracy: 0.9120
Epoch 5/200
342/342 [==============================] - 0s 1ms/step - loss: 0.6529 - accuracy: 0.9134 - val_loss: 0.3224 - val_accuracy: 0.9103
Epoch 6/200
342/342 [==============================] - 0s 1ms/step - loss: 0.6340 - accuracy: 0.9111 - val_loss: 0.2999 - val_accuracy: 0.9094
Epoch 7/200
342/342 [==============================] - 0s 1ms/step - loss: 0.6223 - accuracy: 0.9072 - val_loss: 0.4060 - val_accuracy: 0.8975

Epoch 58/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4423 - accuracy: 0.9013 - val_loss: 0.2551 - val_accuracy: 0.8991
Epoch 59/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4392 - accuracy: 0.9026 - val_loss: 0.2379 - val_accuracy: 0.8998
Epoch 60/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4370 - accuracy: 0.8999 - val_loss: 0.2569 - val_accuracy: 0.8982
Epoch 61/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4296 - accuracy: 0.9018 - val_loss: 0.2253 - val_accuracy: 0.9064
Epoch 62/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.9016 - val_loss: 0.2727 - val_accuracy: 0.8933
Epoch 63/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4317 - accuracy: 0.9004 - val_loss: 0.2902 - val_accuracy: 0.8881
Epoch 64/200
342/342 [==============================] - 0s 1ms/step - loss: 0.4293 - accuracy: 0.9023 - val_loss: 0.3399 - val_accuracy:

Epoch 115/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3623 - accuracy: 0.9203 - val_loss: 0.2582 - val_accuracy: 0.9127
Epoch 116/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3314 - accuracy: 0.9220 - val_loss: 0.1922 - val_accuracy: 0.9293
Epoch 117/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3355 - accuracy: 0.9226 - val_loss: 0.1745 - val_accuracy: 0.9356
Epoch 118/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3397 - accuracy: 0.9183 - val_loss: 0.1909 - val_accuracy: 0.9288
Epoch 119/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3306 - accuracy: 0.9219 - val_loss: 0.2156 - val_accuracy: 0.9230
Epoch 120/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.9175 - val_loss: 0.2497 - val_accuracy: 0.8993
Epoch 121/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3226 - accuracy: 0.9216 - val_loss: 0.2754 - val_ac

342/342 [==============================] - 0s 1ms/step - loss: 0.3103 - accuracy: 0.9250 - val_loss: 0.2033 - val_accuracy: 0.9277
Epoch 172/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3040 - accuracy: 0.9256 - val_loss: 0.2058 - val_accuracy: 0.9228
Epoch 173/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3052 - accuracy: 0.9225 - val_loss: 0.3232 - val_accuracy: 0.8727
Epoch 174/200
342/342 [==============================] - 0s 1ms/step - loss: 0.3053 - accuracy: 0.9241 - val_loss: 0.2203 - val_accuracy: 0.9176
Epoch 175/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2956 - accuracy: 0.9260 - val_loss: 0.2021 - val_accuracy: 0.9319
Epoch 176/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2945 - accuracy: 0.9254 - val_loss: 0.2512 - val_accuracy: 0.9043
Epoch 177/200
342/342 [==============================] - 0s 1ms/step - loss: 0.2908 - accuracy: 0.9247 - val_loss: 0.2200 - val_accuracy: 0.9213

In [29]:
# Get model accuracy 
scores = model2.evaluate(test_features, test_labels)
print('\n')
print('accuracy=',scores[1])

# Get predictions from model
output = model2.predict_classes(test_features)

# Show confusion matrix
y_actu = pd.Series(test_labels, name='Actual')
y_pred = pd.Series(np.ndarray.flatten(output), name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

num_positives =  (np.count_nonzero(y_actu))
num_negatives = y_actu.size - num_positives

TN = df_confusion[0][0]
TP = df_confusion[1][1]
FN = df_confusion[0][1]
FP = df_confusion[1][0]

precision = TP/(TP + FP)
recall = TP/(TP + FN)
specificity = TN/(TN + FP)
f1 = 2*((precision*recall)/(precision+recall))
print("Num Fraud: {}".format(num_positives))

print("Precision: ", precision)
print("Recall/Sensitivity: ", recall)
print("Specificity: ", specificity)
print("F1 Score: ", f1)

167/167 [==============================] - 0s 464us/step - loss: 0.2190 - accuracy: 0.9242


accuracy= 0.9241573214530945
Predicted     0    1
Actual              
0.0        4578  331
1.0          74  357
Num Fraud: 431
Precision:  0.5188953488372093
Recall/Sensitivity:  0.8283062645011601
Specificity:  0.932572825422693
F1 Score:  0.6380697050938338
